## Sources:

Zip code link: https://gis-mdc.opendata.arcgis.com/datasets/fee863cb3da0417fa8b5aaf6b671f8a7/explore

## Imports

In [69]:
import pandas as pd
import geopandas as gpd
import altair as alt
import json
import folium
from folium.plugins import FeatureGroupSubGroup
import os

## Data read-in

In [2]:
df_HPI_Gain = pd.read_csv('CoreLogicData - HPI Gain by Zip Code in Miami Dade, FL.csv')

df_HPI_Forecast = pd.read_csv('CoreLogicData - HPI Projections by Year and Zip Code.csv')

gdf = gpd.read_file('Miami_Dade_Zip_Code.geojson', geometry='geometry')

In [3]:
df_HPI_Gain = df_HPI_Gain.drop(columns=['2018\n December', '2019\n December', '2020\n December',
       '2021\n December', '2022\n December', '2023\n December',])

In [4]:
df = pd.merge(df_HPI_Gain,df_HPI_Forecast,on='Zip Code')

In [5]:
# Now, you can check for valid geometries
gdf_clean = gdf[gdf.geometry.notnull()]

In [6]:
# Note: gpd.GeoDataFrame does have the 'is_valid' attribute, but it applies to the 'geometry' Series
gdf_valid = gdf_clean[gdf_clean.geometry.is_valid]

## Data Prepare

In [7]:
df = df.rename(columns={'Zip Code':'zip_code'})

In [8]:
gdf = pd.merge(df, gdf_valid, left_on='zip_code',right_on='ZIP')

In [9]:
gdf.columns

Index(['zip_code', '5 Year HPI Change', '1 year forecast (23-24)',
       '2 year forecast (23-25)', '3 year forecast (23-26)',
       '4 year forecast (23-27)', '5 year forecast (23-28)', 'OBJECTID',
       'PZIPCODEID', 'ZIP', 'ZIPCODE', 'GlobalID', 'Shape__Area',
       'Shape__Length', 'geometry'],
      dtype='object')

In [10]:
gdf = gpd.GeoDataFrame(gdf)

# Filter out GeometryCollection and other unsupported geometries using geom_type
gdf = gdf[gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])]

In [11]:
gdf = gdf.drop(columns=['OBJECTID','PZIPCODEID','ZIP','ZIPCODE','GlobalID','Shape__Area','Shape__Length'])

In [12]:
gdf.to_file('HPI_Miami-Dade.geojson', driver='GeoJSON')

In [20]:
# # Initialize the map
# m = folium.Map(location=[25.7617, -80.1918], zoom_start=10)

# for column in gdf.columns.drop(['geometry','zip_code']):
#     # Create a FeatureGroup for each column
#     fg = folium.FeatureGroup(name=column)
    
#     # Convert GeoDataFrame filtered by column to GeoJSON
#     geojson = gdf[['geometry', column]].dropna().to_json()
    
#     # Create a GeoJson layer
#     folium.GeoJson(
#         geojson,
#         name=column,
#         style_function=lambda feature: {
#             'fillColor': 'green' if feature['properties'][column] > 50 else 'red',
#             'color': 'black',
#             'weight': 1,
#             'fillOpacity': 0.6,
#         },
#         tooltip=folium.GeoJsonTooltip(fields=['zip_code', column])
#     ).add_to(fg)
    
#     # Add the FeatureGroup to the map
#     fg.add_to(m)

# # Add layer control
# folium.LayerControl().add_to(m)

# # Display map
# m

In [23]:
gdf.columns

Index(['zip_code', '5 Year HPI Change', '1 year forecast (23-24)',
       '2 year forecast (23-25)', '3 year forecast (23-26)',
       '4 year forecast (23-27)', '5 year forecast (23-28)', 'geometry'],
      dtype='object')

In [31]:
import folium

# Initialize the map centered around Miami-Dade County
m = folium.Map(location=[25.7617, -80.1918], zoom_start=10)

# Define a function to create a style_function based on a property
def create_style_function(property_name):
    def style_function(feature):
        # Default fillColor
        fill_color = '#ffff00'  # Bright yellow for visibility
        
        try:
            # Assuming the property value is a string that represents a float
            value = float(feature['properties'][property_name])
            if value > 0.1:  # Adjust threshold as necessary
                fill_color = '#00ff00'  # Green
            else:
                fill_color = '#ff0000'  # Red
        except (ValueError, KeyError):
            # If the value cannot be converted to float or key doesn't exist
            # The default fill_color will be used
            pass
        
        return {
            'fillColor': fill_color,
            'color': 'black',  # Outline color
            'weight': 1,  # Outline weight
            'fillOpacity': 0.6,
        }
    return style_function


# List of properties you want to toggle as layers
properties = ['5 Year HPI Change', '1 year forecast (23-24)',
       '2 year forecast (23-25)', '3 year forecast (23-26)',
       '4 year forecast (23-27)', '5 year forecast (23-28)']  # Replace with your actual property names

# Add a GeoJson layer for each property
for property_name in properties:
    geojson_layer = folium.GeoJson(
        'HPI_Miami-Dade.geojson',
        name=property_name,
        style_function=create_style_function(property_name),
        tooltip=folium.GeoJsonTooltip(fields=['zip_code', property_name])
    ).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map
# m.save('index.html')
m

In [53]:
import folium
import branca.colormap as cm
import geopandas as gpd

# Load your GeoDataFrame
gdf = gpd.read_file('HPI_Miami-Dade.geojson')

gdf = gdf[gdf['geometry'].is_valid]

# Initialize the Folium map with a center point
m = folium.Map(location=[25.7617, -80.1918], zoom_start=10)

# Define a function to create a colormap and style function based on the column
def add_layer(gdf, column_name, map_object):
    # Determine the range of your data values for the specific column
    min_value = gdf[column_name].min()
    max_value = gdf[column_name].max()
    
    # Create a linear colormap
    colormap = cm.linear.YlOrRd_09.scale(min_value, max_value)
    colormap.caption = column_name  # Set a caption for the color scale bar
    
    # Define the style function using the colormap
    def style_function(feature):
        value = feature['properties'][column_name]
        return {
            'fillColor': colormap(value) if value is not None else 'transparent',
            'color': 'black',  # Outline color
            'weight': 1,  # Outline weight
            'fillOpacity': 0.6,
        }
    
    # Create a GeoJson layer with the style function and add it to the map
    geojson_layer = folium.GeoJson(
        gdf[['zip_code','geometry', column_name]].dropna().to_json(),
        name=column_name,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=['zip_code', column_name], aliases=['ZIP Code:', column_name])
    )
    
    # Add the GeoJson layer to a FeatureGroup
    fg = folium.FeatureGroup(name=column_name)
    geojson_layer.add_to(fg)
    
    # Add the FeatureGroup to the map object
    fg.add_to(map_object)
    
    # Add the colormap to the map
    colormap.add_to(map_object)

# Loop over the columns you want to visualize as separate layers
for column in ['5 Year HPI Change', '1 year forecast (23-24)',
       '2 year forecast (23-25)', '3 year forecast (23-26)',
       '4 year forecast (23-27)', '5 year forecast (23-28)']:  # Replace with your actual column names
    add_layer(gdf, column, m)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m

In [67]:
gdf['5 year forecast (23-28)'].min()*100

20.3

In [68]:
gdf['5 year forecast (23-28)'].max()*100

26.400000000000002

## Map URL Snagger

In [70]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDadeZipCodesHPIforecast
